In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
import os as os
import pickle

In [2]:
file_list = os.listdir()
file_list.remove('Generating_tensors.ipynb')
file_list.remove('.ipynb_checkpoints')
file_list.remove('Xs_physical.npy')
file_list.remove('ys_physical.npy')
file_list.remove('Feature_selection_regression.ipynb')

In [3]:
# Transform the datasets into Xs and Ys
def TransformSamples3 (sample1, sample2, sample3):
    sample1 = sample1.T.to_numpy()[1:]
    sample2 = sample2.T.to_numpy()[1:]
    sample3 = sample3.T.to_numpy()[1:]
    x = np.append(sample1, sample2, axis=0).astype('float32')
    x = np.append(x, sample3, axis=0).astype('float32')
    y = np.array([0]*len(sample1) + [1]*len(sample2)+[2]*len(sample3))
    return x, y

In [4]:
healthy_group_filename = []
moderate_group_filename = []
severe_group_filename = []
for i in file_list:
    if 'Healthy' in i:
        healthy_group_filename.append(i)
        continue
    if 'Moderate' in i:
        moderate_group_filename.append(i)
        continue
    if 'Severe' in i:
        severe_group_filename.append(i)
        continue

In [16]:
healthy_group = []
moderate_group = []
severe_group = []
for i in healthy_group_filename:
    healthy_group.append(pd.read_csv(i, index_col='Unnamed: 0'))
for i in moderate_group_filename:
    moderate_group.append(pd.read_csv(i, index_col='Unnamed: 0'))
for i in severe_group_filename:
    severe_group.append(pd.read_csv(i, index_col='Unnamed: 0'))

In [17]:
severe_group

[         Genename       Counts    Sample
 0            B9D2   563.571300  Severe 3
 1            JAG2    20.375315  Severe 3
 2          DNASE1   453.483670  Severe 3
 3            TRMU  1041.491200  Severe 3
 4            RRP9   737.336000  Severe 3
 ...           ...          ...       ...
 17201       FOXF1     0.000000  Severe 3
 17202  AC092316.1     0.000000  Severe 3
 17203   FOXP1-AS1     7.122574  Severe 3
 17204     ANKRD61     0.000000  Severe 3
 17205   TNKS2-AS1    25.048235  Severe 3
 
 [17206 rows x 3 columns],
          Genename       Counts    Sample
 0            B9D2   664.738460  Severe 1
 1            JAG2     6.733430  Severe 1
 2          DNASE1   858.351870  Severe 1
 3            TRMU  1388.805500  Severe 1
 4            RRP9   931.412300  Severe 1
 ...           ...          ...       ...
 17201       FOXF1     0.000000  Severe 1
 17202  AC092316.1     0.000000  Severe 1
 17203   FOXP1-AS1    17.126390  Severe 1
 17204     ANKRD61     3.549876  Severe 1
 1720

In [18]:
genes = healthy_group[0]['Genename'].values

In [19]:
from sklearn.preprocessing import MinMaxScaler

In [20]:
scaler = MinMaxScaler()
Xs = []
ys = []
for i in healthy_group:
    Xs.append(scaler.fit_transform(np.log1p(i['Counts'].values).reshape(-1,1)).flatten())
    ys.append(0)
for i in moderate_group:
    Xs.append(scaler.fit_transform(np.log1p(i['Counts'].values).reshape(-1,1)).flatten())
    ys.append(1)
for i in severe_group:
    Xs.append(scaler.fit_transform(np.log1p(i['Counts'].values).reshape(-1,1)).flatten())
    ys.append(2)
Xs = np.array(Xs)
ys = np.array(ys)

In [21]:
Xs_train, Xs_test = train_test_split(Xs, random_state=7)
ys_train, ys_test = train_test_split(ys, random_state=7)

In [52]:
np.save('ys_severity', ys)

# Random Forest

In [22]:
from sklearn.model_selection import cross_val_score, ShuffleSplit
from sklearn.ensemble import RandomForestClassifier

In [23]:
rf = RandomForestClassifier(max_depth=10, n_jobs=10)
rf_result_list = []
rf.fit(Xs_train,ys_train)
rf_result = pd.DataFrame({'Gene index':np.arange(len(genes)),
              'Importance':rf.feature_importances_}).sort_values(by='Importance')
rf_result_selected = rf_result[rf_result['Importance']!=0]
rf_result_list.append(rf_result_selected)
rf.score(Xs_test, ys_test)

0.75

In [27]:
# An accuracy improvement!
rf.fit(Xs_train[:,rf_result_selected['Gene index']], ys_train)
rf.score(Xs_test[:,rf_result_selected['Gene index']], ys_test)

0.9166666666666666

In [28]:
# Save the RF model
with open ('RF.pickle', 'wb') as file:
    pickle.dump(rf, file)

In [8]:
with open ('RF.pickle', 'rb') as file:
    rf_load = pickle.load(file)

/home/aimmunelab/anaconda3/envs/harristf/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/aimmunelab/anaconda3/envs/harristf/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [9]:
rf_load.feature_importances_

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.00085452, 0.        , 0.        , 0.        ,
       0.        , 0.00260355, 0.00338837, 0.00150097, 0.00053922,
       0.        , 0.        , 0.00036464, 0.        , 0.        ,
       0.00158204, 0.        , 0.0008381 , 0.0004558 , 0.00585516,
       0.        , 0.00489472, 0.00195268, 0.00372519, 0.        ,
       0.00264662, 0.00452612, 0.        , 0.        , 0.        ,
       0.00311998, 0.        , 0.        , 0.        , 0.00065089,
       0.01216542, 0.        , 0.        , 0.        , 0.        ,
       0.00298193, 0.        , 0.        , 0.0014399 , 0.0032457 ,
       0.00062147, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00166387, 0.        , 0.        ,
       0.00080409, 0.        , 0.        , 0.        , 0.00435028,
       0.        , 0.00286624, 0.        , 0.        , 0.00130178,
       0.00150768, 0.00049699, 0.        , 0.        , 0.01464

In [30]:
rf_load.score(Xs[:,rf_result_selected['Gene index']], ys)

0.9777777777777777

In [31]:
rf.predict_proba(Xs_test[:,rf_result_selected['Gene index']])

array([[0.94, 0.01, 0.05],
       [0.64, 0.06, 0.3 ],
       [0.95, 0.01, 0.04],
       [0.93, 0.03, 0.04],
       [0.77, 0.08, 0.15],
       [0.63, 0.12, 0.25],
       [0.72, 0.04, 0.24],
       [0.44, 0.13, 0.43],
       [0.04, 0.62, 0.34],
       [0.13, 0.05, 0.82],
       [0.8 , 0.01, 0.19],
       [0.19, 0.2 , 0.61]])

In [35]:
pd.DataFrame({'Gene index':rf_result_selected['Gene index'].values,
              'Gene name':genes[rf_result_selected['Gene index'].values]}).to_csv('RF_selected_genes.csv')

In [31]:
selected_genes = []
for i in rf_result_selected['Gene index']:
    selected_genes.append(genes[i])
rf_result_selected['Gene name'] = selected_genes
rf_result_selected.to_csv('RF_selected_genes.csv')

/tmp/ipykernel_142432/2856623931.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rf_result_selected['Gene name'] = selected_genes


In [35]:
rf_result_selected.sort_values(by='Importance',ascending=False).to_csv('RF_selected_genes.csv')

# Recursive feature elimination

In [44]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

In [47]:
estimator = SVC(C=3, kernel='linear')
selector = RFE(estimator)

In [48]:
selector = selector.fit(Xs, ys)

In [49]:
selected_genes = np.unique(np.append(np.unique(selector.ranking_[:1000,]),rf_result_selected['Gene index'].values))

In [50]:
selected_genes = pd.DataFrame({'Gene index':selected_genes,
                               'Gene name': genes[selected_genes]})

In [51]:
selected_genes

,Gene index,Gene name
0,1,JAG2
1,14,CNOT4
2,43,HIST1H4C
3,62,S100A8
4,67,ABCD1
...,...,...
1029,16946,FAM215A
1030,16967,RHCE
1031,17012,MTSS1L
1032,17030,NTN5


In [88]:
selected_genes.to_csv('selected_genes.csv')

In [5]:
np.savetxt('Severity_related'pd.read_csv('selected_genes.csv', index_col='Unnamed: 0')['Gene name'])

0          JAG2
1         RPS16
2          CFL1
3        OGFRL1
4       TMEM245
         ...   
963    ARHGEF37
964     SLC15A5
965       ASB14
966    ARHGAP36
967      ZNF564
Name: Gene name, Length: 968, dtype: object

In [41]:
pd.read_table('RF_selected_genes_converted.txt')[]

,From,To,Species,Gene Name
0,OXTR,5021,Homo sapiens,oxytocin receptor(OXTR)
1,CERK,64781,Homo sapiens,ceramide kinase(CERK)
2,MYT1L,23040,Homo sapiens,myelin transcription factor 1 like(MYT1L)
3,WIPF2,147179,Homo sapiens,WAS/WASL interacting protein family member 2(W...
4,GABPB1,2553,Homo sapiens,GA binding protein transcription factor beta s...
...,...,...,...,...
362,RAB13,5872,Homo sapiens,"RAB13, member RAS oncogene family(RAB13)"
363,SPRY2,10253,Homo sapiens,sprouty RTK signaling antagonist 2(SPRY2)
364,BCOR,54880,Homo sapiens,BCL6 corepressor(BCOR)
365,FSIP1,161835,Homo sapiens,fibrous sheath interacting protein 1(FSIP1)
